<a href="https://colab.research.google.com/github/Pki03/DL_grpassgn/blob/main/dl_grpassgn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
import pandas as pd

df = pd.read_csv("/content/Haryanvi_Sentiment_Dataset (2).csv")

In [ ]:
df.head()


,text,sentiment
0,खेत में नई फसल लगाई सै,neutral
1,इस बरखा ने सारा अनाज सड़ा दियो,negative
2,आज मंदिर में भीड़ ज्यादा सै,neutral
3,मेरी गाय बीमार हो गई सै,negative
4,दादी के हाथ की रोटी का कोई जोड़ नई,positive


In [ ]:
df["sentiment"].value_counts()


,count
sentiment,
positive,1216
negative,1138
neutral,762


In [ ]:
from sklearn.preprocessing import LabelEncoder

X = df["text"]
y = df["sentiment"]

le = LabelEncoder()
labels = le.fit_transform(y)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, labels, test_size=0.2, random_state=42
)

print("Classes:", set(labels))


Classes: {np.int64(0), np.int64(1), np.int64(2)}


In [ ]:
import numpy as np
num_classes = len(np.unique(labels))



In [ ]:
import tensorflow as tf
from tensorflow import keras
from transformers import TFBertForSequenceClassification, BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

def tokenize_data(texts, labels):
    tokenized = tokenizer(
        texts.tolist(),
        padding='max_length',
        truncation=True,
        max_length=128,  # Efficient for Punjabi text
        return_tensors='tf'
    )
    return {
        'input_ids': tokenized['input_ids'],
        'attention_mask': tokenized['attention_mask']
    }, tf.one_hot(labels, depth=num_classes)



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices(tokenize_data(X_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices(tokenize_data(X_test, y_test))

batch_size = 64

train_dataset = train_dataset.shuffle(len(X_train)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)



In [ ]:
model = TFBertForSequenceClassification.from_pretrained(
    'bert-base-multilingual-cased',
    num_labels=num_classes
)

model.compile(
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=2e-5),
    metrics=["accuracy"]
)



model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.fit(train_dataset, epochs=5, validation_data=test_dataset)

Epoch 1/5
39/39 [==============================] - 117s 2s/step - loss: 0.7431 - accuracy: 0.6766 - val_loss: 0.2556 - val_accuracy: 0.9215
Epoch 2/5
39/39 [==============================] - 63s 2s/step - loss: 0.1782 - accuracy: 0.9478 - val_loss: 0.1002 - val_accuracy: 0.9776
Epoch 3/5
39/39 [==============================] - 63s 2s/step - loss: 0.0786 - accuracy: 0.9795 - val_loss: 0.0738 - val_accuracy: 0.9808
Epoch 4/5
39/39 [==============================] - 63s 2s/step - loss: 0.0485 - accuracy: 0.9856 - val_loss: 0.0915 - val_accuracy: 0.9776
Epoch 5/5
39/39 [==============================] - 63s 2s/step - loss: 0.0346 - accuracy: 0.9928 - val_loss: 0.0686 - val_accuracy: 0.9840


In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)
print(f"\nTest Accuracy: {test_acc:.4f}")



10/10 [==============================] - 5s 501ms/step - loss: 0.0686 - accuracy: 0.9840

Test Accuracy: 0.9840


In [ ]:
def predict_sentiment(text):
    inputs = tokenizer(
        text,
        padding='max_length',
        truncation=True,
        max_length=128,
        return_tensors='tf'
    )
    outputs = model(inputs)
    logits = outputs.logits
    probabilities = tf.nn.softmax(logits, axis=-1)
    predicted_class = tf.argmax(probabilities, axis=-1).numpy()[0]
    return le.inverse_transform([predicted_class])[0], probabilities.numpy()[0]


In [ ]:
sample_text = "इब्बै तेरे तै बात करन का भी मन ना सै, तू तो बिलकुल ही बदल ग्या सै।"
sentiment, confidence = predict_sentiment(sample_text)

print(f"\nText: {sample_text}")
print(f"Predicted Sentiment: {sentiment}")
print(f"Confidence: {confidence}")



Text: इब्बै तेरे तै बात करन का भी मन ना सै, तू तो बिलकुल ही बदल ग्या सै।
Predicted Sentiment: negative
Confidence: [0.9946727  0.00158978 0.00373754]
